In this kernel I decided to try PicoSAT for few reasons.

First I think it's important for this competition to learn as much as possible about states that don't have previous state. That is what is called [Garden of Eden](https://en.wikipedia.org/wiki/Garden_of_Eden_(cellular_automaton)) at least in original version of Game of Life with unbounded board.

One of the brilliant examples is [Flower of Eden](https://www.conwaylife.com/wiki/Flower_of_Eden) - the smallest known rotationally symmetric Garden of Eden.
![Flower of Eden](https://www.conwaylife.com/w/images/7/78/Orphan5.png)

More precisely configuration above is called *orphan* - a subset of cells such as no matter what other cell states are resulting state will be a Garden of Eden. This means that any 25x25 board that contains this or any other orphan configuration inside it, will not have previous state as well.

More over, for bounded 25x25 board, that we are given, most of the possible board states don't have previous state simply because:
* there are many board states like empty board with variety of predecessors
* total number of states stays the same on each step = 2^625

Every Garden of Eden is a "dead end" in solving each task, so we should learn patterns of these states and avoid them as much as possible.

Some research of Gardens of Eden led to solution in 2016 of [Conway's Grandfather problem](https://www.conwaylife.com/wiki/Grandfather_problem) and this is where PicoSAT comes into play as a tool that was used to solve the problem, along with finding
> "A father and grandfather, but no great-grandfather" pattern[7], and a "father, grandfather and great-grandfather, but no great-great-grandfather" pattern

While link above can give a lot of useful information on its own, one of the take aways is that PicoSAT can be an efficient tool for this competition if used wisely.

As a python wrapper I'm going to use [pycosat](https://pypi.org/project/pycosat/) - first that I found. It is very easy to use, which is good for a starter Kernel.


I took and modified few helper functions (pretty few) from other public notebooks, so don't be surprized, and I apollogize if I didn't mention original authors.

In [ ]:
import numpy as np
import pandas as pd
from numba import njit, prange
from scipy.signal import convolve2d
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pycosat
import random

window = np.ones((3, 3))

def plot_3d(solution_3d: np.ndarray, title, size=3, max_cols=10, has_target=False):
    N = len(solution_3d)
    if N <= 0:
        return
    cols = min(N, max_cols)
    rows = (N - 1) // max_cols + 1
    plt.figure(figsize=(cols*size, rows*size))
    plt.suptitle(title)
    for t in range(N):
        board = solution_3d[t]
        plt.subplot(rows, cols, t + 1)
        plt.imshow(board, cmap='binary')
        plt.title('target' if t == 0 and has_target else f'state {t}')
    plt.show()

def csv_to_numpy_list(df) -> np.ndarray:
    return df[[ f'stop_{n}' for n in range(25**2) ]].values.reshape(-1,25,25)

def life_step(X: np.ndarray):
    C = convolve2d(X, window, mode='same', boundary='wrap')
    return (C == 3) | (X & (C == 4))

For SAT solvers we only need Test data: there is nothing to train and we always know if solution is correct.

In [ ]:
sample_submission_df = pd.read_csv('../input/conways-reverse-game-of-life-2020/sample_submission.csv', index_col='id')
test_df = pd.read_csv('../input/conways-reverse-game-of-life-2020/test.csv', index_col='id')
deltas = test_df['delta'].values
boards = csv_to_numpy_list(test_df)

Ok, let's try out some of the hepler functions above and look at some sample test board.

In [ ]:
for index in tqdm(range(1), total=1):
    board = np.tile(boards[index], (2, 1, 1))
    board[1] = life_step(board[0])
    plot_3d(board, 'sample board')

Ok, finally we are getting to the solver itself.

Looks like pycosat requires input in [Conjunctive normal form](https://en.wikipedia.org/wiki/Conjunctive_normal_form) and does not support weights, optional clauses or native cardinality constraints. It can present some problems later when we would want to limit the search space or prefer some solutions to another. But for now the simpler the better.

While trying to build CNF expression for Game of Life, I found this article, that helped me to reduce number of clauses and variables a lot:
https://sat-smt.codes/SAT_SMT_by_example.pdf (Pages 501-504)

In [ ]:
SIZE = 25

empty_board = np.zeros((SIZE,SIZE), dtype=bool)

# Computes number of variables.
# I negated variable states after some testing, because solver
# seems to prefer True as default state for solution variables,
# while I want cells to be empty whenever possible
def v(c):
    return -(SIZE * c[0] + c[1] + 1)

def dead_clauses(res, c, x):
    # if cell is dead, there was not exactly 3 alive neighbours (56 clauses)
    for i1 in range(0, 6):
        for i2 in range(i1+1, 7):
            for i3 in range(i2+1, 8):
                a = [v(x[i]) for i in range(8)]
                a[i1], a[i2], a[i3] = -a[i1], -a[i2], -a[i3]
                res.append(a)
    # if cell is dead and was alive, was not 2-3 alive neighbours (28 clauses)
    for i1 in range(0, 7):
        for i2 in range(i1+1, 8):
            a = [v(x[i]) if i < 8 else -v(c) for i in range(9)]
            a[i1], a[i2] = -a[i1], -a[i2]
            res.append(a)

def live_clauses(res, c, x):
    # if cell is alive, there was less then 4 alive neighbours (70 clauses)
    for i1 in range(0, 5):
        for i2 in range(i1+1, 6):
            for i3 in range(i2+1, 7):
                for i4 in range(i3+1, 8):
                    #from each 4 at least 1 was dead
                    res.append([-v(x[i1]), -v(x[i2]), -v(x[i3]), -v(x[i4])])
    # if cell is alive and was dead, there was more than 2 alive (less than 6 dead) neighbours (28 clauses)
    for i1 in range(0, 7):
        for i2 in range(i1+1, 8):
            a = [v(x[i]) if i < 8 else v(c) for i in range(9) if i != i1 and i != i2]
            res.append(a)
    # if cell is alive, there was more than 1 alive (less than 7 dead) neighbours (8 clauses)
    for i1 in range(0, 8):
        a = [v(x[i]) for i in range(8) if i != i1]
        res.append(a)

def board_clauses(board, use_opt = True):
    res, opt1, opt2 = [], [], []
    for i in range(SIZE):
        for j in range(SIZE):
            x = [((i + k % 3 - 1) % SIZE, (j + k // 3 - 1) % SIZE) for k in range(9) if k != 4]
            if board[i,j]:
                live_clauses(res, (i, j), x) # 106 clauses
            else:
                dead_clauses(res, (i, j), x) # 84 clauses
                if use_opt:
                    y = [((i + k % 5 - 2) % SIZE, (j + k // 5 - 2) % SIZE) for k in range(25)]
                    if sum(board[ii,jj] for ii,jj in y) < 1: # No alive neighbours
                        res.append([-v((i, j))]) # Very dead space MUST stay dead! (1 clause)
                    elif sum(board[ii,jj] for ii,jj in x) < 1: # No alive neighbours
                        opt1.append([-v((i, j))]) # Dead space should stay dead! (1 clause)
                    elif sum(board[ii,jj] for ii,jj in x) < 2: # Too few alive neighbours
                        opt2.append([-v((i, j))]) # Dead space should stay dead! (1 clause)

    return res, opt1, opt2

Depth(delta)-1 tasks can be solved relatively easy:

In [ ]:
N = len(deltas)
score = 0
for n in tqdm(range(N), total=N):
    clauses, opt1, opt2 = board_clauses(boards[n], use_opt = False)
    solution = pycosat.solve(clauses)
    if isinstance(solution, str):
        print(f'{n} not solved!')
        continue
    board = np.array(solution[:SIZE**2]) < 0
    sample_submission_df.loc[test_df.index[n]] = 1 * board
    board = life_step(board.reshape(SIZE,SIZE))
    d = np.sum(board ^ boards[n])
    score += d / 625
print(score/N)

It takes just a bit more than half a second to solve delta-1 task in average.

Next, I've built step-by-step solver for individual tasks, with ability to gradualy exclude optional clauses and fallback to previous step when there is no solution.

Let's try it out for first tasks.

In [ ]:
N = 1
for n in tqdm(range(N), total=N):
    T = min(deltas[n], 3)
    board = np.tile(empty_board, (T+1, 1, 1))
    board[0] = boards[n]
    solvers = [None for _ in range(T)]
    opt = [None for _ in range(T)]
    os = [0 for _ in range(T)]
    oe = [0 for _ in range(T)]
    t = 0
    while 0 <= t and t < T:
        if solvers[t] is None:
            clauses, opt1, opt2 = board_clauses(board[t])
            solution = pycosat.solve(clauses)
            if not isinstance(solution, str):
                if t == T - 1:
                    print(t, '!!', end=" ")
                    t += 1
                    board[t] = np.array(solution[:SIZE**2]).reshape(SIZE,SIZE) < 0
                    continue
                else:
                    print(t, '??', end=" ")
                    random.shuffle(opt1)
                    random.shuffle(opt2)
                    opt[t] = opt1 + opt2
                    os[t] = len(opt[t])+1
                    oe[t] = len(opt[t])+1
                    solvers[t] = pycosat.itersolve(clauses + opt[t])
                    print(len(opt[t]), end=" ")
        try:
            solution = next(solvers[t])
            if oe[t] - os[t] > 1:
                os[t] = (os[t]+oe[t])//2
                solvers[t] = pycosat.itersolve(clauses + opt[t][:(os[t]+oe[t])//2])
                print((os[t]+oe[t])//2, end=" ")
            else:
                print(t, '++', end=" ")
                t += 1
                board[t] = np.array(solution[:SIZE**2]).reshape(SIZE,SIZE) < 0
        except Exception as err:
            if solvers[t] is not None and (os[t]+oe[t])//2 > 0:
                if os[t] == oe[t]:
                    os[t] = 0
                    oe[t] = len(opt[t])+1
                elif oe[t] - os[t] > 1:
                    oe[t] = (os[t]+oe[t])//2
                else:
                    os[t] -= 1
                    oe[t] -= 1
                solvers[t] = pycosat.itersolve(clauses + opt[t][:(os[t]+oe[t])//2])
                print((os[t]+oe[t])//2, end=" ")
            else:
                print(t, '--', end=" ")
                solvers[t] = None
                opt[t] = None
                t -= 1
    if t == T:
        sample_submission_df.loc[test_df.index[n]] = 1 * board[T].ravel()
    plot_3d(board, index)

So it does solve some tasks already!

Obviously performance is not good enough and a lot of optimization needs to be done for deep backwards search (3+ steps).

But depth-1 tasks can be solved relatively easy:

Finally, let's save resulting submission

In [ ]:
sample_submission_df.to_csv("submission.csv", index=True)

In [ ]:
submission_df = pd.read_csv('./submission.csv', index_col='id')